# Feed Forward Regression


### Importing packages


In [ ]:
import os

import numpy as np
import pandas as pd
import torch
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from utils import plot_loss_epoch, plot_mse, plot_pred_target

### Loading data


In [ ]:
bidding_area = 1

df = pd.read_csv(f"data/NO{1}.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"])
df.set_index("timestamp", inplace=True)

df.head()

### Add historic temperature


In [ ]:
df["past_consumption"] = df["consumption"].shift(1)
df.rename(
    columns={"consumption": "next_consumption", "past_consumption": "consumption"},
    inplace=True,
)
df = df[["temperature", "consumption", "next_consumption"]]
df.dropna(inplace=True)

df.head()

### Adding time, day and year features


In [ ]:
df["time_of_day"] = df.index.hour
df["day_of_week"] = df.index.dayofweek
df["day_of_year"] = df.index.dayofyear

df.head()

### Splitting data


In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2, shuffle=False)
df_train, df_val = train_test_split(df_train, test_size=0.1)

### Normalizing training data


In [ ]:
train_mean = df_train.mean()
train_std = df_train.std()

df_train = (df_train - train_mean) / train_std
df_val = (df_val - train_mean) / train_std
df_test = (df_test - train_mean) / train_std

In [ ]:
df_test.info()

### Converting to tensors


In [ ]:
target_train = torch.tensor(df_train["next_consumption"].values.astype(np.float32))
features_train = torch.tensor(
    df_train.drop(columns="next_consumption").values.astype(np.float32)
)

target_val = torch.tensor(df_val["next_consumption"].values.astype(np.float32))
features_val = torch.tensor(
    df_val.drop(columns="next_consumption").values.astype(np.float32)
)

target_test = torch.tensor(df_test["next_consumption"].values.astype(np.float32))
features_test = torch.tensor(
    df_test.drop(columns="next_consumption").values.astype(np.float32)
)

### Create pytorch datasets and data loaders


In [ ]:
train_tensor = TensorDataset(features_train, target_train)
train_loader = DataLoader(train_tensor, batch_size=64, shuffle=True)

val_tensor = TensorDataset(features_val, target_val)
val_loader = DataLoader(val_tensor, batch_size=64, shuffle=False)

test_tensor = TensorDataset(features_test, target_test)
test_loader = DataLoader(
    test_tensor, batch_size=25, shuffle=False
)  # 25 because one day has 24 hours

### Creating model


In [ ]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


# Define model
class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(5, 24),
            nn.ReLU(),
            nn.Linear(24, 12),
            nn.ReLU(),
            nn.Linear(12, 6),
            nn.Linear(6, 1),
        )

    def forward(self, x):
        x = self.net(x)
        return x


model = FeedForward().to(device)
model

### Loss function and optimizer


In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

### Training


In [96]:
epochs = 20

losses = []
losses_show = []
val_losses = []

for e in range(epochs):
    model.train()
    epoch_loss = []

    for batch, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y.view(-1, 1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        epoch_loss.append(loss.item())

        if len(losses) > 20:
            losses_show.append(sum(losses[-20:]) / 20)
        else:
            losses_show.append(losses[-1])

    val_loss = []
    for batch, (X, y) in enumerate(val_loader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y.view(-1, 1))
        val_loss.append(loss.item())

    avg_val_loss = sum(val_loss) / len(val_loss)
    print(f"Epoch {e+1}, Val Loss: {avg_val_loss}")

    val_losses.append(avg_val_loss)

models_directory = "models"

if not os.path.exists(models_directory):
    os.makedirs(models_directory)

torch.save(model.state_dict(), "models/fnn.pth")

KeyboardInterrupt: 

### Plotting


In [ ]:
plot_loss_epoch(losses_show, val_losses)

### Testing using Step-by-Step Strategy


In [ ]:
model.eval()

with torch.no_grad():
    predictions = []

    for batch, (X, y) in enumerate(test_loader):
        X, y = X.to(device), y.to(device)

        preds = []
        i = 0
        pred = 0
        for f, t in zip(X, y):
            if i > 0:
                f[1] = pred
                pred = model(f)
                preds.append(pred.item())
            elif i == 0:
                pred = model(f)
                preds.append(pred.item())
            i += 1
        predictions.append([preds, y])

### Plotting predictions and targets


In [ ]:
plot_pred_target(predictions, train_mean, train_std, 200)

### Plotting Mean Squared Error with Deviation


In [ ]:
plot_mse(predictions, train_mean, train_std)